## 三大法人買賣超日報

In [11]:
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import pandas as pd

In [12]:
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload={
'qdate':'105/05/13',
'select2':'24',
'sorting':'by_issue'    
}

In [13]:
def getTWDate(dt):
    year = int(dt.strftime('%Y')) - 1911
    monthdate = dt.strftime('%m/%d')
    ymd = '{}/{}'.format(year, monthdate)
    return ymd

def getTradingVolume(date):
    payload['qdate'] = getTWDate(date)
    res = requests.post(url, data=payload)    
    soup = bs(res.text)
    tb = soup.select('#tbl-sortable')[0]
    dfs = pd.read_html(tb.prettify('utf-8'), encoding='utf-8')
    stockdfs = dfs[0]
    stockdfs['date'] = date.date()
    return dfs[0]

In [14]:
import requests

currentdate = datetime.now()
dfall = []
for i in range(1,3):
    ymd = currentdate - timedelta(days = i)
    df = getTradingVolume(ymd)
    dfall.append(df)

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [15]:
stockdf = pd.concat(dfall)

## SQLite 操作
不用安裝好像也可以直接用

In [3]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
con.close()

In [6]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()
cur.execute('SELECT SQLITE_VERSION()')
data = cur.fetchone()
print data
con.close()

(u'3.6.21',)


In [7]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS PhoneAddress") 
cur.execute("CREATE TABLE PhoneAddress(phone CHAR(10) PRIMARY KEY, address TEXT, name TEXT unique, age INT NOT NULL)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0912173381','United State','Jhon Doe',53)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0928375018','Tokyo Japan','MuMu Cat',6)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0957209108','Taipei','Richard',29)") 
con.commit()
cur.execute("SELECT phone,address FROM PhoneAddress") 

data = cur.fetchall()
print data
con.close()

[(u'0912173381', u'United State'), (u'0928375018', u'Tokyo Japan'), (u'0957209108', u'Taipei')]


In [8]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()

cur.execute("SELECT phone,address FROM PhoneAddress") 

data = cur.fetchone()
print data[0], data[1]

data = cur.fetchall()
print data
con.close()

0912173381 United State
[(u'0928375018', u'Tokyo Japan'), (u'0957209108', u'Taipei')]


In [9]:
a = (1,2) #tuple方法比較少
b = [1,2]
dir(b)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [16]:
stockdf.head()

,證券 代號,證券 名稱,外資 買進股數,外資 賣出股數,外資 買賣超股數,投信 買進股數,投信 賣出股數,投信 買賣超股數,自營商 買賣超股數,自營商 買進股數 (自行買賣),自營商 賣出股數 (自行買賣),自營商 買賣超股數 (自行買賣),自營商 買進股數 (避險),自營商 賣出股數 (避險),自營商 買賣超股數 (避險),三大法人 買賣超股數,date
0,2408.0,南亞科,18079000.0,4097000.0,13982000.0,0.0,0.0,0.0,329000.0,317000.0,17000.0,300000.0,154000.0,125000.0,29000.0,14311000.0,2016-05-13
1,3474.0,華亞科,9560000.0,4665000.0,4895000.0,0.0,0.0,0.0,113000.0,201000.0,30000.0,171000.0,0.0,58000.0,-58000.0,5008000.0,2016-05-13
2,2449.0,京元電,8513011.0,4469755.0,4043256.0,58000.0,490000.0,-432000.0,472000.0,0.0,107000.0,-107000.0,1266000.0,687000.0,579000.0,4083256.0,2016-05-13
3,3413.0,京鼎,529000.0,24000.0,505000.0,0.0,0.0,0.0,-1000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,504000.0,2016-05-13
4,3016.0,嘉晶,440000.0,91000.0,349000.0,0.0,0.0,0.0,-12000.0,2000.0,14000.0,-12000.0,0.0,0.0,0.0,337000.0,2016-05-13


In [17]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
stockdf.to_sql(name = 'trading_volume', index=False, con = con, if_exists="replace")
#con.commit()
con.close()

C:\Anaconda2\lib\site-packages\pandas\core\generic.py:1160: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [18]:
#stockdf.head()
#len(stockdf.columns)
stockdf.columns = ['id', 'name', 'buy', 'sale', '4','5', '6', '7' '8','9','10','11','12', '13', '14', '15', '16', 'date']

In [19]:
with lite.connect('finance2.sqlite') as con:
    stockdf.to_sql(name = 'trading_volume', index=False, con = con, if_exists="replace")

In [20]:
import pandas as pd
with lite.connect('finance2.sqlite') as con:
    df2 = pd.read_sql_query('select * from trading_volume', con)
df2.head()

,id,name,buy,sale,4,5,6,78,9,10,11,12,13,14,15,16,date
0,2408.0,南亞科,18079000.0,4097000.0,13982000.0,0.0,0.0,0.0,329000.0,317000.0,17000.0,300000.0,154000.0,125000.0,29000.0,14311000.0,2016-05-13
1,3474.0,華亞科,9560000.0,4665000.0,4895000.0,0.0,0.0,0.0,113000.0,201000.0,30000.0,171000.0,0.0,58000.0,-58000.0,5008000.0,2016-05-13
2,2449.0,京元電,8513011.0,4469755.0,4043256.0,58000.0,490000.0,-432000.0,472000.0,0.0,107000.0,-107000.0,1266000.0,687000.0,579000.0,4083256.0,2016-05-13
3,3413.0,京鼎,529000.0,24000.0,505000.0,0.0,0.0,0.0,-1000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,504000.0,2016-05-13
4,3016.0,嘉晶,440000.0,91000.0,349000.0,0.0,0.0,0.0,-12000.0,2000.0,14000.0,-12000.0,0.0,0.0,0.0,337000.0,2016-05-13


In [21]:
df2.columns

Index([u'id', u'name', u'buy', u'sale', u'4', u'5', u'6', u'78', u'9', u'10',
       u'11', u'12', u'13', u'14', u'15', u'16', u'date'],
      dtype='object')

In [22]:
import pandas as pd
with lite.connect('finance2.sqlite') as con:
    df2 = pd.read_sql_query(\
        u"select id, name from trading_volume where buy > 307000", con)
df2

,id,name
0,2408.0,南亞科
1,3474.0,華亞科
2,2449.0,京元電
3,3413.0,京鼎
4,3016.0,嘉晶
5,8016.0,矽創
6,2401.0,凌陽
7,2388.0,威盛
8,2337.0,旺宏
9,2379.0,瑞昱


In [25]:
df2.to_excel('test.xlsx')#匯出EXECEL
#tab鍵可以提示
#alt+enter 執行並增加一行

In [27]:
df3 = pd.read_excel('test.xlsx')

In [28]:
df3

,id,name
0,2408,南亞科
1,3474,華亞科
2,2449,京元電
3,3413,京鼎
4,3016,嘉晶
5,8016,矽創
6,2401,凌陽
7,2388,威盛
8,2337,旺宏
9,2379,瑞昱


## 2330　102年第1季 IFRS 合併財務報表預覽

In [29]:
import requests
url = 'http://mops.twse.com.tw/server-java/t164sb01'
payload = {
'step':'1',
'CO_ID':'2330',
'SYEAR':'2013',
'SSEASON':'1',
'REPORT_ID':'C'
}
res = requests.post(url, data= payload)
print res.encoding
res.encoding = 'big5'
print res.encoding

ISO-8859-1
big5


In [32]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text)
tb = soup.select('.result_table')[0]
#print tb


In [33]:
import pandas as pd
dfs = pd.read_html(tb.prettify('utf-8'), encoding = 'utf-8', header=0, skiprows=0)

In [34]:
stockdf = dfs[0]
stockdf['stockno'] = '2330'
import sqlite3 as lite
with lite.connect('finance2.sqlite') as con:
    stockdf.to_sql('ifrs', con = con, if_exists='replace')

## 三大法人買賣明細資訊

In [ ]:
Document(Doc):
requests    ------------> document: <html> data </html>
100 * 3
    
AJAX(XHR):
requests    ------------> document: <html></html>
requests    ------------> javascript: $('html').html(data)
requests    ------------> data: data
2 + 1 + 100 * 1

In [ ]:
XML:
<employee>
    <name> qoo </name>
    <age> 10 <age>
</employee>

JSON:
{employee:{name:qoo, age:10}}

In [35]:
import requests
res = requests.get('http://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&se=EW&t=D&d=105/05/10&_=1463283795803')
print res

<Response [200]>


In [36]:
#print res.text
import json
jd = json.loads(res.text)
#print jd['aaData']

In [37]:
import pandas as pd
df = pd.DataFrame(jd['aaData'])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,006201,寶富櫃,0,0,0,0,0,0,"-45,000",0,0,0,0,"45,000","-45,000","-45,000"
1,1258,F-其祥,"3,000",0,"3,000",0,0,0,"-1,000",0,"1,000","-1,000",0,0,0,"2,000"
2,1259,安心,"14,000",0,"14,000",0,0,0,0,0,0,0,0,0,0,"14,000"
3,1264,德麥,"4,000","1,000","3,000",0,0,0,0,0,0,0,0,0,0,"3,000"
4,1565,精華,"148,000","136,000","12,000",0,0,0,"-1,000",0,0,0,"1,000","2,000","-1,000","11,000"


ctrl+M,M可以切換成命令模式並且變成文件編輯



## 三大法人交易資訊



In [38]:
import requests
url = 'http://www.taifex.com.tw/chinese/3/7_12_1.asp'
payload = {
'DATA_DATE_Y':'2016',
'DATA_DATE_M':'5',
'DATA_DATE_D':'12',
'syear':'2016',
'smonth':'5',
'sday':'12',
'datestart':'2016/5/12'    
}
res = requests.post(url, data = payload)
res.encoding='utf-8'
print res

<Response [200]>


In [42]:
#print res.text

In [44]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text)
tb = soup.select('.table_f')[0]


In [ ]:
?pd.read_html

In [45]:
import pandas as pd
dfs = pd.read_html(tb.prettify('utf-8'), \
            encoding = 'utf-8', skiprows=[0,1], header=0)
stockdf = dfs[0]

In [46]:
stockdf.columns = ['身份別','多方口數','多方契約金額',\
                   '空方口數','空方契約金額',\
                   '多空淨額口數','多空淨額契約金額']
stockdf['date'] = '2016/5/12' 
stockdf

,身份別,多方口數,多方契約金額,空方口數,空方契約金額,多空淨額口數,多空淨額契約金額,date
0,自營商,288779,39991,266375,46217,22404,-6226,2016/5/12
1,投信,4753,7459,4011,6302,742,1157,2016/5/12
2,外資,174311,89034,168547,86050,5764,2984,2016/5/12
3,合計,467843,136484,438933,138569,28910,-2085,2016/5/12


## 證交所三大法人csv檔抓取

In [47]:
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload = {
'download':'csv',
'qdate':'105/05/13',
'select2':'01',
'sorting':'by_issue'    
}
res = requests.post(url , payload)
res.encoding= 'big5'

with open('20160513.csv', 'w') as f:
    f.write(res.text.encode('utf-8'))

In [48]:
import pandas as pd
df = pd.read_csv('20160513.csv', names=["證券代號","證券名稱","外資買進股數","外資賣出股數",\
                "外資買賣超股數","投信買進股數","投信賣出股數","投信買賣超股數","自營商買賣超股數",\
                "自營商買進股數(自行買賣)","自營商賣出股數(自行買賣)","自營商買賣超股數(自行買賣)",\
                "自營商買進股數(避險)","自營商賣出股數(避險)","自營商買賣超股數(避險)","三大法人買賣超股數"])
df = df.dropna()
df = df.drop(1)
df.head()

,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
2,"=""1108""",幸福,"2,000","2,000",0,0,0,0,0,0,0,0,0,0,0,0
3,"=""1110""",東泥,"13,000","16,000","-3,000",0,0,0,0,0,0,0,0,0,0,"-3,000"
4,"=""1104""",環泥,"61,000","118,000","-57,000",0,0,0,"-9,000",0,"9,000","-9,000",0,0,0,"-66,000"
5,"=""1103""",嘉泥,0,"79,000","-79,000",0,0,0,0,0,0,0,0,0,0,"-79,000"
6,"=""1102""",亞泥,"2,308,000","3,899,000","-1,591,000",0,0,0,"221,000",0,"84,000","-84,000","305,000",0,"305,000","-1,370,000"


In [49]:
df.columns

Index([u'證券代號', u'證券名稱', u'外資買進股數', u'外資賣出股數', u'外資買賣超股數', u'投信買進股數',
       u'投信賣出股數', u'投信買賣超股數', u'自營商買賣超股數', u'自營商買進股數(自行買賣)', u'自營商賣出股數(自行買賣)',
       u'自營商買賣超股數(自行買賣)', u'自營商買進股數(避險)', u'自營商賣出股數(避險)', u'自營商買賣超股數(避險)',
       u'三大法人買賣超股數'],
      dtype='object')

In [50]:
df['證券代號']= df['證券代號'].str.strip('=').str.strip('\"') #移除不需要的符號
df

,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
2,1108,幸福,"2,000","2,000",0,0,0,0,0,0,0,0,0,0,0,0
3,1110,東泥,"13,000","16,000","-3,000",0,0,0,0,0,0,0,0,0,0,"-3,000"
4,1104,環泥,"61,000","118,000","-57,000",0,0,0,"-9,000",0,"9,000","-9,000",0,0,0,"-66,000"
5,1103,嘉泥,0,"79,000","-79,000",0,0,0,0,0,0,0,0,0,0,"-79,000"
6,1102,亞泥,"2,308,000","3,899,000","-1,591,000",0,0,0,"221,000",0,"84,000","-84,000","305,000",0,"305,000","-1,370,000"
7,1101,台泥,"1,008,000","5,384,000","-4,376,000",0,0,0,"350,000","194,000","2,000","192,000","258,000","100,000","158,000","-4,026,000"


In [51]:
a = [['david', 1], ['mary', 2]]
df1 = pd.DataFrame(a)
df1

,0,1
0,david,1
1,mary,2


In [52]:
b = [{'name':'david', 'id':1}, {'name':'mary', 'id':2}]
df2 = pd.DataFrame(b)
df2

,id,name
0,1,david
1,2,mary


In [53]:
import requests
headers = {
'X-Requested-With':'XMLHttpRequest',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'    
}
url = 'https://statementdog.com/pick/pick_ajax/?type=1&idx1=15&tm1=2&op1=1&val1=15&_=1463292903371'
res = requests.get(url, headers = headers)

#print res.text

In [54]:
import pandas as pd
import json
jd = json.loads(res.text)
#print jd[1]
df = pd.DataFrame(jd[1])
df.head()

,data,id,nm,price,roi,ticker
0,"[{u'tm': 2, u'val': 18.45, u'idx': 15}]",18,福懋油,30.45,"[44.9, 132.1, 101.1]",1225
1,"[{u'tm': 2, u'val': 21.5, u'idx': 15}]",19,佳格,77.10,"[1.6, -19.9, -8.0]",1227
2,"[{u'tm': 2, u'val': 25.3, u'idx': 15}]",22,大統益,77.90,"[-0.8, 47.9, 47.7]",1232
3,"[{u'tm': 2, u'val': 24.54, u'idx': 15}]",27,F-鮮活,118.50,"[-45.2, 72.8, None]",1256
4,"[{u'tm': 2, u'val': 17.6, u'idx': 15}]",28,F-其祥,38.25,"[33.8, 56.1, None]",1258


In [55]:
import requests
headers = {
#'User-Agent':'qoo'
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'    
}
res = requests.get('http://ck101.com/', headers = headers)
#print res.text

## gogofun

In [56]:
import requests
headers = {
'Referer':'http://www.gogofund.com/convch/fund/sectorperform.asp',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'    
}
url = 'http://www.gogofund.com/convch/fund/forframe/sectorperform_sql.asp?FundTypeID=EG&PerformMoney=TWD&Prd=0&O=5&P=1&r=0.22726785669738292'
res = requests.get(url, headers = headers)
#print res.text

## Selenium

 建立腳本

In [7]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re

from bs4 import BeautifulSoup as bs
import pandas as pd

driver = webdriver.Firefox()
driver.implicitly_wait(30)
base_url = "http://www.gogofund.com/"
   
driver.get(base_url + "/convch/index.asp")
driver.find_element_by_css_selector(u"a[title=\"各基金類型績效排行\"]").click()
soup = bs(driver.page_source)
tb = soup.select('#sectorperform_output table')[0]
dfs = pd.read_html(tb.prettify('utf-8'), encoding = 'utf-8')
dfs[0]

driver.find_element_by_xpath(u"//input[@value='下一頁']").click()
soup = bs(driver.page_source)
tb = soup.select('#sectorperform_output table')[0]
dfs = pd.read_html(tb.prettify('utf-8'), encoding = 'utf-8')
dfs[0]
driver.close()